In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.stats import shapiro 
import scipy.signal as spsig
import scipy.ndimage as spim
import cupyx.scipy.ndimage as cpim
import cupy as cp
from scipy.signal import butter, filtfilt
import scipy.stats

%matplotlib widget

sys.path.insert(0, '../../../drlib')
import drlib as dr

def freq2idx(freq, freqs):
    return int(freq/freqs[-1]*len(freqs))

In [ ]:
def lam(nu, nu0, p0 = None, nBins=None, v0=220, vlab=233, offset = 0):
    # Work in MHz, this will explode for large frequency values (ie 100e6 Hz)
    # velocity args in km/s
    # vlab has yearly modulation between about 220 and 255, see fig 3a in Gramolin
    # output is normilized to p0
    if nBins is not None:
        # find the index of nu0 in nu array
        idx = np.abs(nu-nu0).argmin()

        # calculate the slice start and end indices
        start_idx = max(0, idx - nBins//2)
        end_idx = min(len(nu), idx + nBins//2)
        nu = nu[start_idx:end_idx]

    c       = 299792 #km/s
    nu0     = nu0 + offset
    beta    = np.zeros_like(nu) # initialise beta with zeros

    mask    = (nu > nu0) # create a mask of values where nu > nu0
    # calculate beta only for values where nu > nu0
    beta[mask] = (2*c*vlab)/(v0**2) * np.sqrt((2*(nu[mask]-nu0))/(nu0))
    beta    = np.minimum(beta, 100)
    a       = (2*c**2)/(np.sqrt(np.pi)*v0*vlab*nu0)
    b       = -((beta**2*v0**2)/(4*vlab**2)) - (vlab**2/v0**2) 

    lamArr  = a*np.exp(b)*np.sinh(beta)
    if p0 is not None:
        outPSD = p0/np.sum(lamArr) * lamArr #normilize to p0
    else:
        outPSD = lamArr

    return nu, outPSD 


 # normal values
nu0             = 399
freqs           = np.linspace(0, 400, 2**23)
nBins           = 100

p0              = None
lamOut          = lam(freqs, nu0, p0, nBins=nBins, offset=-10e-6)
freqs           = lamOut[0]
lamArr          = lamOut[1]

noise           = np.random.normal(0, 1, nBins) *0
spec            = lamArr+noise
plt.close('all')
plt.figure()
plt.title('Rayleigh line shape (normilized to P0 total)')

plt.plot(freqs, spec)
#plt.scatter(freqs, lamArr)
plt.xlabel('Frequency (MHz)')
plt.ylabel('PSD (Watts/bin)')
plt.legend(loc = 'upper left')

In [ ]:
lamOut = lam(freqs, nu0, nBins=100)[1]
conv = np.convolve(lamOut, spec, mode='same')
print(len(conv))
print(len(freqs))

print(f"SNR spec: {spec.max()/scipy.stats.median_abs_deviation(spec)}")
print(f"SNR conv: {conv.max()/scipy.stats.median_abs_deviation(conv)}")

plt.close('all')
plt.figure()
plt.plot(freqs, conv)
plt.show()

In [ ]:
def gen_nu_steps(freqs, startFreq, percent):
    freq    = startFreq
    nuStepList  = []
    while freq < freqs[-1]:
        #if freq > freqs:
        #    nu0Arr.append(400)
        nuStepList.append(freq)
        endFreq = freq + freq*percent/100
        freq = endFreq
    nuStepList.append(freqs[-1]) 
    return nuStepList

def freq2idx(freq, freqs):
    return int(freq/freqs[-1]*len(freqs))



#test data
freqs       = np.linspace(0,400,2**23)
nu0         = 250.1
p0          = 100
lamOut      = lam(freqs, nu0, p0, nBins=None, offset=-10e-6)
lamArr      = lamOut[1]


#chirp spec
np.random.seed(0)  # for reproducibility
frequency = np.linspace(0, 400, 2**23)  # frequency range
undulation = spsig.chirp(frequency, 1/800, frequency[-1], 1/.1) + 100
noise = np.random.normal(10, .01, frequency.shape)  # Gaussian noise
#noise[2**20:2**20+10] = 10.05
spec = (undulation * noise + lamArr)  # noisy spectrum

#normal noise
spec = np.random.normal(0,1,len(freqs)) + lamArr



#search parameters
startFreq   = 45
percent     = 150
overlap     = 100 #bins

nuStepList  = gen_nu_steps(freqs, startFreq, percent)
outputConv  = []
outputFreqs = []
#change template over frequency range
for i, nu in enumerate(nuStepList[0:-1]):
    idxStart        = freq2idx(nuStepList[i], freqs) - overlap
    nu0             = nuStepList[i] + (nuStepList[i+1] - nuStepList[i])/2
    if nuStepList[i+1] == freqs[-1]:
        idxStop     = freq2idx(nuStepList[i+1], freqs) - 10
    else:
        idxStop     = freq2idx(nuStepList[i+1], freqs) + overlap
    lamOut      = lam(freqs, nu0, p0=1, nBins=100)
    subConv     = np.convolve(lamOut[1], spec[idxStart:idxStop], mode='same')[overlap:-overlap]
    subFreqs    = freqs[idxStart+overlap:idxStop-overlap]
    
    outputConv.append(subConv)
    outputFreqs.append(subFreqs)



    '''print(f'idx start : {idxStart}')
    print(f'f0      : {nu0}')
    print(f'idx stop  : {idxStop}')
    print()'''

outputConv = np.hstack(outputConv)
outputFreqs = np.hstack(outputFreqs)

print(len(outputConv))
print(len(outputFreqs))
print(outputFreqs)


plt.close('all')

plt.figure()
plt.plot(freqs, spec)
plt.show()

plt.figure()
plt.plot(outputFreqs, outputConv)
plt.show()




In [ ]:
#get data
#real data
spectrum = np.load('/drBiggerBoy/run1p4/plottingSpec/chB_avg_W_switch0.npy')[1:]
spectrum[2**22:2**22+10] *= 1
frequency = np.linspace(0,400,2**23)

freqStart       = 45
freqStop        = 301
idxStart        = freq2idx(freqStart, frequency)
idxStop         = freq2idx(freqStop, frequency)

#test data
testDataFlag = 1
if testDataFlag:
    if 1:
        frequency   = outputFreqs
        spectrum    = outputConv
    if 0:
        np.random.seed(0)  # for reproducibility
        frequency = np.linspace(0, 400, 2**23)  # frequency range

        freqStart       = .5
        freqStop        = 9.5
        idxStart        = freq2idx(freqStart, frequency)
        idxStop         = freq2idx(freqStop, frequency)

        undulation = 10*np.sin(frequency * 100) + frequency*10 +100 # sinusoidal signal
        noise = np.random.normal(10, .01, frequency.shape)  # Gaussian noise
        noise[2**20:2**20+6] = 11.2
        spectrum = (undulation * noise)  # noisy spectrum

    if 0:
        #chirp    
        np.random.seed(0)  # for reproducibility
        frequency = np.linspace(0, 400, 2**23)  # frequency range
        undulation = spsig.chirp(frequency, 1/800, frequency[idxStop], 1/.1) + 100
        noise = np.random.normal(10, .01, frequency.shape)  # Gaussian noise
        noise[2**20:2**20+10] = 10.05
        spectrum = (undulation * noise)  # noisy spectrum

    if 0: #bode plot test
        np.random.seed(0)  # for reproducibility
        frequency = np.linspace(0, 2**23, 2**23)  # frequency range
        spectrum = spsig.chirp(frequency, 1/100000, frequency[-1], 1)


plt.figure()
plt.plot(frequency, spectrum)
plt.show()


In [ ]:
#low pass and divide by "fit"
plt.close('all')
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import scipy.stats


def normalize_spectrum(spectrum):
    """
    Function to normalize a spectrum by its mean
    """
    # Low-pass filter design
    cutoff = 40000  # choose a suitable cutoff frequency
    nyq = 0.5 * len(spectrum)  # Nyquist frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(1, normal_cutoff, btype='low', analog=False)
    
    # Apply the low-pass filter
    spectrum_filtered = filtfilt(b, a, spectrum)

    # Normalize the spectrum by the mean of the filtered spectrum
    normalized_spectrum = spectrum / (spectrum_filtered)

    return normalized_spectrum, spectrum_filtered

# Normalize the test spectrum
normalized_spectrum, spectrum_filtered = normalize_spectrum(spectrum)

normalized_spectrum_analysis = normalized_spectrum[idxStart:idxStop]
spectrum_filtered_analysis  = spectrum_filtered[idxStart:idxStop]
freqs_analysis              = frequency[idxStart:idxStop]

print(f"norm spec mean {np.mean(normalized_spectrum_analysis)}")
print(f"norm spec std {np.std(normalized_spectrum_analysis)}")



In [ ]:
#median filter and divide by "fit"

import scipy.ndimage as spim
import cupyx.scipy.ndimage as cpim
import cupy as cp

def normalize_spectrum(spectrum):
    """
    Function to normalize a spectrum by its mean
    """
    # Low-pass filter design
    spectrumGpu = cp.array(spectrum)


    medFiltSpecGpu = cpim.median_filter(
        spectrumGpu,
        size=(50),
        origin=0
    )
    spectrum_filtered = medFiltSpecGpu.get()
    
    
    # Low-pass filter design
    cutoff = 40000  # choose a suitable cutoff frequency
    nyq = 0.5 * len(spectrum_filtered)  # Nyquist frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(6, normal_cutoff, btype='low', analog=False)
    
    if 1:
        # Apply the low-pass filter
        spectrum_filtered = filtfilt(b, a, spectrum_filtered)
    

    # Normalize the spectrum by the mean of the filtered spectrum
    normalized_spectrum = spectrum  /spectrum_filtered

    return normalized_spectrum, spectrum_filtered


# Normalize the test spectrum
normalized_spectrum, spectrum_filtered = normalize_spectrum(spectrum)

normalized_spectrum_analysis = normalized_spectrum[idxStart:idxStop]
spectrum_filtered_analysis  = spectrum_filtered[idxStart:idxStop]
freqs_analysis              = frequency[idxStart:idxStop]

print(f"norm spec mean {np.mean(normalized_spectrum_analysis)}")
print(f"norm spec std {np.std(normalized_spectrum_analysis)}")
print(scipy.stats.normaltest(normalized_spectrum_analysis))
print(scipy.stats.normaltest(noise))

In [ ]:
#SG filter and divide by "fit"

import scipy.signal as spsig

def normalize_spectrum(spectrum):
    """
    Function to normalize a spectrum by its mean
    """

    spectrum_filtered = spectrum
    if 1: #median filter
        spectrumGpu = cp.array(spectrum)
        medFiltSpecGpu = cpim.median_filter(
            spectrumGpu,
            size=(50),
            origin=0
        )
        spectrum_filtered = medFiltSpecGpu.get()
    if 1:
        # SG filter
        spectrum_filtered = spsig.savgol_filter(
            spectrum_filtered,
            window_length=(51),
            polyorder=1
        )
    
    if 0:
        # Low-pass filter design
        cutoff = 40000  # choose a suitable cutoff frequency
        nyq = 0.5 * len(spectrum_filtered)  # Nyquist frequency
        normal_cutoff = cutoff / nyq
        b, a = butter(4, normal_cutoff, btype='low', analog=False)
        
        # Apply the low-pass filter
        spectrum_filtered = filtfilt(b, a, spectrum_filtered)
    

    # Normalize the spectrum by the mean of the filtered spectrum
    normalized_spectrum = spectrum  /spectrum_filtered

    return normalized_spectrum, spectrum_filtered


# Normalize the test spectrum
normalized_spectrum, spectrum_filtered = normalize_spectrum(spectrum)

normalized_spectrum_analysis = normalized_spectrum[idxStart:idxStop]
spectrum_filtered_analysis  = spectrum_filtered[idxStart:idxStop]
freqs_analysis              = frequency[idxStart:idxStop]

print(f"norm spec mean {np.mean(normalized_spectrum_analysis)}")
print(f"norm spec std {np.std(normalized_spectrum_analysis)}")
print(scipy.stats.normaltest(normalized_spectrum_analysis))
print(scipy.stats.normaltest(noise))

In [ ]:
plt.close('all')
if 1:
    # Plot the test spectrum
    plt.close('all')
    plt.figure(figsize=(10, 5))
    plt.plot(frequency, spectrum, label='Original spectrum')
    plt.plot(freqs_analysis, spectrum_filtered_analysis, label='filtered spectrum', color='red')
    plt.xlabel('Frequency')
    plt.ylabel('Power')
    plt.title('Test Spectrum')
    plt.grid(True)
    plt.legend(loc = 'upper left')
    plt.show()

if testDataFlag and 0:
    # plot undulation (H) vs filtered <H>
    plt.figure(figsize=(10, 5))
    plt.plot(freqs_analysis, 10*undulation[idxStart:idxStop] - spectrum_filtered_analysis, label='<H>', color='blue')
    plt.xlabel('Frequency')
    plt.ylabel('Power')
    plt.title('H - <H> diff')
    plt.grid(True)
    plt.legend(loc = 'upper left')
    plt.show()

if testDataFlag and 0:
    # plot undulation (H) vs filtered <H>
    plt.figure(figsize=(10, 5))
    plt.plot(freqs_analysis, 10*undulation[idxStart:idxStop], label='H', color='blue')
    plt.plot(freqs_analysis, spectrum_filtered_analysis, label='<H>', color='red')
    plt.xlabel('Frequency')
    plt.ylabel('Power')
    plt.title('H and <H>')
    plt.grid(True)
    plt.legend(loc = 'upper left')
    plt.show()


if 1:
    # Normalized Spectrum
    nSigma = 5
    oneSigMad = 1.4826*scipy.stats.median_abs_deviation(normalized_spectrum_analysis)

    plt.figure(figsize=(10, 5))
    plt.plot(freqs_analysis, (normalized_spectrum_analysis-1)/oneSigMad, label='Normalized spectrum')
    plt.plot((freqs_analysis[0], freqs_analysis[-1]), (nSigma, nSigma), label = f'{nSigma} Sigma Limit')
    plt.xlabel('Frequency')
    plt.ylabel('Normalized Power')
    plt.title('Normalized Spectrum')
    plt.grid(True)
    plt.legend(loc = 'lower left')
    plt.show()

if testDataFlag and 0:
    # Z score, noise vs norm
    plt.figure(figsize=(10, 5))
    plt.plot(freqs_analysis, scipy.stats.zscore(normalized_spectrum_analysis), label='Normalized spectrum', color='blue')
    plt.plot(frequency, scipy.stats.zscore(noise), label='Noise', color='red', alpha = 0.5)
    plt.xlabel('Frequency')
    plt.ylabel('Normalized Power')
    plt.title('Z score, noise vs norm')
    plt.grid(True)
    plt.legend(loc = 'upper left')
    plt.show()

if testDataFlag and 0:
    # Plot the z score diff spectrum
    plt.figure(figsize=(10, 5))
    plt.plot(freqs_analysis, scipy.stats.zscore(normalized_spectrum_analysis) - scipy.stats.zscore(noise)[idxStart:idxStop], label='Normalized spectrum', color='blue')
    plt.xlabel('Frequency')
    plt.ylabel('Normalized Power')
    plt.title('Z score diff')
    plt.grid(True)
    plt.legend(loc = 'upper left')
    plt.show()

if 0:
    # Plot the 5 sigma limit
    plt.figure(figsize=(10, 5))
    plt.plot(frequency, spectrum, label='Spectrum', color='blue')
    plt.plot(freqs_analysis, (1+1.4826*5*scipy.stats.median_abs_deviation(normalized_spectrum_analysis))*spectrum_filtered_analysis, label='limit', color='red')
    plt.xlabel('Frequency')
    plt.ylabel('Normalized Power')
    plt.title('Spec and 5 sigma limit')
    plt.grid(True)
    plt.legend(loc = 'upper left')
    plt.show()

if 0:

    # calculate common bin edges
    min_bin_edge = min(np.min(scipy.stats.zscore(normalized_spectrum_analysis)), np.min(scipy.stats.zscore(noise[idxStart:idxStop])))
    max_bin_edge = max(np.max(scipy.stats.zscore(normalized_spectrum_analysis)), np.max(scipy.stats.zscore(noise[idxStart:idxStop])))

    bin_edges = np.linspace(min_bin_edge, max_bin_edge, 500)  # this creates 50 bins

    # plot histograms
    plt.figure(figsize=(10, 5))
    plt.title('Z score')
    plt.hist(scipy.stats.zscore(normalized_spectrum_analysis), bins=bin_edges, label='Normalized Spectrum')
    plt.hist(scipy.stats.zscore(noise[idxStart:idxStop]), bins=bin_edges, alpha=0.5, label='Original Noise')
    plt.legend(loc='upper left')
    plt.semilogy()
print(len(normalized_spectrum_analysis))
print(len(noise[idxStart:idxStop]))

In [ ]:
#Generate a fake background
numPoints               = 2**18
freqs                   = np.linspace(0, 400, numPoints)

demoSpec                = np.random.normal(10, 1, numPoints)
demoSpec[numPoints//2]  *= 2
demoSpec                *= np.sin(1-freqs/900)


plt.close('all')
plt.plot(freqs, demoSpec)

plt.figure()
plt.plot(freqs, demoSpec/np.sin(1-freqs/900))


In [ ]:
import numpy as np

class AnalysisPipeline:
    """
    A class to perform a dark photon search on an averaged spectrum

    Attributes
    ----------
    spec            : arr
        Power spectrum to search for dark photon signal
    freqs           : arr
        Array of frequenies that corosponds to spec
    analysisSpec    : arr
        Spectrum that is being worked on

    Methods
    -------
    high_pass_filter(fcNumBins=int,
        order=int):
        Pre-filters the spectrum to remove DC offset and large scale undulations

    optimal_filter(
        ):
        Performs a convolution of the input against a template array
        Calls gen_template

    gen_template(
        ):
        Generates a template function array
    
    rolling_STD(window=int,
        filt=bool,
        fcNumBins=int,
        order=int,
        numProc=int (opt.),
        mode=str
        ):
        Calculates rolling 1 STD threshold using an optional function

    find_candidates(CL=float,
        ):
        Calculates a threshold such that there is a (1-CL)% chance
        that a bin is above that threshold.
    """
    def __init__(self,
        spec, 
        freqs
        ):

        self.spec = spec
        self.freqs = freqs
        self.analysisSpec = np.zeros(spec)

    def high_pass_filter(self):
        '''
        Performs basic buttersworth filtering of a spectrum.
            Parameters:
                fcNumBins (int) : Corner "frequency" in bins
                order (int)     : Order of filter
        '''
        pass

    def optimal_filter(self):
        """
        Implement optimal filtering/convolution here
        """
        pass

    def rolling_STD(self):
        '''
        Computes a rolling STD on the input spectrum. 
            Parameters:
                window (int)    : length of window to compute rolling function on
                filt (bool)     : should the output be filtered. 1 = yes
                fcNumBins (int) : if filt, corner "frequency" (in bins)
                order (int)     : if filt, order of filter
                mode (str)      : how to handle edges
                                'nan pad'
                                    preserve length of input array. Pad ends with nands
                numProc (int)   : number of processers to use
                
        '''
        pass

    def calculate_threshold(self):
        """
        Implement threshold calculation here
        """
        pass

    def run_analysis(self):
        self.high_pass_filter()
        self.optimal_filter()
        self.calculate_rolling_mad()
        self.calculate_threshold()


######## Call this from notebook ########
# Instantiate the analysis pipeline
#analysis = AnalysisPipeline(spectrum, window_size, step_size, num_processes)

# Run the full analysis
#analysis.run_analysis()

In [ ]:
class avgSpecAnalysis:
    '''
    This class takes in an average spectrum and returns a list of candidates.
    There should be some code that runs before this that unpacks the saved data 
    structure and averages together some or all spectra. 
    If candidates are found, it is up to a future analysis to inspect time dependance.
    ####INPUTS####
    freqs: array like
        frequency array
    avgSpec: array like
        Single "master spectrum". All time dependance has been averaged out.

    '''
    def __init__(self,
    freqs,
    avgSpec,

    ):
        self.avgSpec        = avgSpec
        self.freqs          = freqs

    def mad_np(arr
    ):
        median = np.median(arr)
        return np.median(np.abs(arr-median))

    def freq2Idx(self,
        freq, 
        freqsAll
    ):
        return int(freq/freqsAll[-1] * len(freqsAll))

    def idx2freq(self,
        idx,
        freqs
    ):
        return  ((freqs[-1] - freqs[0])/len(freqs) * idx) + freqs[0]
    
    def rollingMadLim(self,
        nSigma      = 1,
        window      = 1
    ):

        rollingMadArr = dr.rolling(self.specFiltered, window, 1, self.mad_np, numProc=48)

        filter_fc           = 30 * window
        filteredMadArr  = dr.filterSpec(rollingMadArr, order = 2, fc_numBins = filter_fc, type = 'lowpass')
        diffFilterMadPadArr = dr.nanPad(filteredMadArr, window)
        rollMadLim      = diffFilterMadPadArr * nSigma * 1.48
        return rollMadLim, specFiltered, nSigma, order, fc_numBins

    def HPF_filter(self,
        order       = 1,
        fc_numBins  = 100,
    ):
        self.specFiltered = dr.filterSpec(spec, fc_numBins=fc_numBins, order=order)


In [ ]:
analysis =  avgSpecAnalysis(freqs,
    demoSpec)

rollingMadLim = analysis.rollingMadLim()

In [ ]:
plt.close('all')
plt.figure()
plt.plot(freqs, 5*rollingMadLim[0])
plt.plot(freqs, rollingMadLim[1])

In [ ]:
specFiltered = dr.filterSpec(demoSpec, fc_numBins=100, order=1)
plt.close('all')
plt.figure()
plt.plot(freqs, specFiltered)